<div style="background-color: #103778; 
            color: white; 
            padding: 10px; 
            text-align: center;
            font-size: 20px;">
    <h1>PARTE II</h1>
</div>

Agora que os dados foram tratados e a Análise Exploratória (EDA) foi concluída, partimos para a etapa de construção do modelo preditivo.

O modelo de previsão de tempo de entrega é um componente estratégico em operações de delivery, pois permite gerenciar as expectativas dos clientes, otimizar a logística e aumentar a fidelização. Estimativas mais precisas contribuem para a construção de confiança, reduzem reclamações e apoiam decisões operacionais, como alocação de entregadores e planejamento de rotas.

Neste projeto, o objetivo é prever um valor numérico contínuo (tempo de entrega). Para esse tipo de problema, a abordagem mais indicada é a utilização de modelos de **Regressão**.

Modelos de Regressão buscam aprender uma função matemática que descreva a relação entre as variáveis independentes por exemplo, distância, tipo de veículo, características do pedido e a variável alvo, tempo de entrega. O desempenho desses modelos é avaliado com base na capacidade de generalizar bem para dados não vistos, minimizando o erro de previsão.

Serão avaliados diferentes algoritmos de Regressão, com características distintas de complexidade e capacidade de modelagem:
- Regressão Linear; 
- Random Forest Regressor; 
- LightGBM;
- XGBoost;
- CatBoost;

Ao final, os modelos serão comparados por meio de métricas de avaliação apropriadas para problemas de regressão, a fim de identificar aquele que apresenta o melhor desempenho para o contexto do problema.

## Importando Bibliotecas e Dataset

In [1]:
# Import das Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import utils.model as ml

import warnings
warnings.filterwarnings('ignore')

/home/rafael/Documentos/Previsão de tempo de entrega/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importanto o dataset de treino
df_treino = pd.read_csv('data/data_modelo.csv', sep = ',')

display(df_treino)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,...,Type_of_order_Buffet,Type_of_order_Drinks,Type_of_order_Meal,Type_of_order_Snack,Avaliacao,vehicle_ratio,mean_distance_driver,mean_distance_order,mean_distance_rating,age_x_rating
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,...,0,0,0,1,3,0.582301,10.506969,9.806712,9.497729,181.3
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,...,0,0,0,1,2,0.335302,11.249514,9.806712,10.181421,153.0
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,...,0,1,0,0,2,0.579375,9.396230,9.710204,10.181421,101.2
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,...,1,0,0,0,3,0.588258,10.821467,9.677144,9.497729,178.6
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,...,0,0,0,1,3,0.335302,9.036788,9.806712,9.497729,147.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41468,1178,RANCHIRES16DEL01,35,4.2,23.371292,85.327872,23.481292,85.437872,Drinks,motorcycle,...,0,1,0,0,2,0.579375,8.766654,9.710204,10.181421,147.0
41469,7C09,JAPRES04DEL01,30,4.8,26.902328,75.794257,26.912328,75.804257,Meal,motorcycle,...,0,0,1,0,3,0.583398,8.361523,9.657065,9.497729,144.0
41470,4F8D,CHENRES08DEL03,30,4.9,13.022394,80.242439,13.052394,80.272439,Drinks,scooter,...,0,1,0,0,3,0.336751,8.983493,9.710204,9.497729,147.0
41471,5EEE,COIMBRES11DEL01,20,4.7,11.001753,76.986241,11.041753,77.026241,Snack,motorcycle,...,0,0,0,1,3,0.582301,9.141467,9.806712,9.497729,94.0


Excluindo as features que não serão usadas

In [3]:
# Elimina as colunas que não serão utilizadas
df_treino = df_treino.drop(['ID', 'Delivery_person_ID', 'Type_of_order', 'Type_of_vehicle', 'rating_categoria'], axis=1)

In [4]:
display(df_treino)

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Time_taken(min),Distance,Type_of_vehicle_bicycle,Type_of_vehicle_electric_scooter,...,Type_of_order_Buffet,Type_of_order_Drinks,Type_of_order_Meal,Type_of_order_Snack,Avaliacao,vehicle_ratio,mean_distance_driver,mean_distance_order,mean_distance_rating,age_x_rating
0,37,4.9,22.745049,75.892471,22.765049,75.912471,24,3.023250,0,0,...,0,0,0,1,3,0.582301,10.506969,9.806712,9.497729,181.3
1,34,4.5,12.913041,77.683237,13.043041,77.813237,33,20.170858,0,0,...,0,0,0,1,2,0.335302,11.249514,9.806712,10.181421,153.0
2,23,4.4,12.914264,77.678400,12.924264,77.688400,26,1.551783,0,0,...,0,1,0,0,2,0.579375,9.396230,9.710204,10.181421,101.2
3,38,4.7,11.003669,76.976494,11.053669,77.026494,21,7.785510,0,0,...,1,0,0,0,3,0.588258,10.821467,9.677144,9.497729,178.6
4,32,4.6,12.972793,80.249982,13.012793,80.289982,30,6.206239,0,0,...,0,0,0,1,3,0.335302,9.036788,9.806712,9.497729,147.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41468,35,4.2,23.371292,85.327872,23.481292,85.437872,33,16.589850,0,0,...,0,1,0,0,2,0.579375,8.766654,9.710204,10.181421,147.0
41469,30,4.8,26.902328,75.794257,26.912328,75.804257,32,1.488910,0,0,...,0,0,1,0,3,0.583398,8.361523,9.657065,9.497729,144.0
41470,30,4.9,13.022394,80.242439,13.052394,80.272439,16,4.654271,0,0,...,0,1,0,0,3,0.336751,8.983493,9.710204,9.497729,147.0
41471,20,4.7,11.001753,76.986241,11.041753,77.026241,26,6.228480,0,0,...,0,0,0,1,3,0.582301,9.141467,9.806712,9.497729,94.0


<div style="background-color: #103778; 
            color: white; 
            padding: 10px; 
            text-align: center;
            font-size: 20px;">
    <h1>Divisão em Treino e Teste</h1>
</div>

In [5]:
# Divisão entre treino e teste
from sklearn.model_selection import train_test_split

# Divisão entre target e o que será usado no modelo
X = df_treino.drop(['Time_taken(min)'], axis=1)
y = df_treino['Time_taken(min)']

# Divisão entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

<div style="background-color: #103778; 
            color: white; 
            padding: 10px; 
            text-align: center;
            font-size: 20px;">
    <h1>1. Treinamento do Modelo</h1>
</div>

In [6]:
df_metricas = ml.train_and_compare_models(X_train, X_test, y_train, y_test)
df_metricas.style.format("{:.3f}", subset = df_metricas.select_dtypes(include="number").columns).background_gradient(cmap='Blues')

,Modelo,MAE,MSE,RMSE,R²,R² Ajustado,MAPE (%),RMSLE
0,Regressão Linear,6.164,60.736,7.793,0.310,0.309,27.121,0.090
1,LightGBM,5.727,53.115,7.288,0.397,0.396,25.053,0.079
2,Random Forest Regressor,5.962,57.767,7.600,0.344,0.343,26.115,0.086
3,XGBoost,5.859,56.103,7.490,0.363,0.362,25.541,0.083
4,CatBoost,5.739,53.551,7.318,0.392,0.391,25.102,0.080


<div style="background-color: #103778; 
            color: white; 
            padding: 10px; 
            text-align: center;
            font-size: 20px;">
    <h1>2. Métricas de Avaliação</h1>
</div>

As métricas de avaliação são fundamentais para medir o desempenho de modelos de **Regressão**, pois permitem quantificar o quão próximas as previsões estão dos valores reais, irei focar em 4 métricas pois julgo serem as mais importantes para avaliar o modelo deste projeto:

- **MAE (Mean Absolute Error)** - Representa a média do valor absoluto das diferenças entre os valores preditos e os valores reais. Seu valor mínimo é 0, indicando previsões perfeitas, e não possui limite superior. Por ser uma métrica em escala linear, o MAE é fácil de interpretar e menos sensível a outliers do que métricas quadráticas.

- **MSE (Mean Squared Error)** - Calcula a média dos erros ao quadrado entre os valores previstos e os valores reais. Assim como o MAE, seu valor mínimo é 0, e quanto maior o valor, pior o desempenho do modelo. Por elevar os erros ao quadrado, o MSE penaliza fortemente erros grandes, sendo bastante sensível a outliers.

- **R² (Coeficiente de Determinação)** - Mede a proporção da variância da variável alvo que é explicada pelo modelo. Seus valores geralmente variam entre 0 e 1, onde valores mais próximos de 1 indicam melhor ajuste. Um R² igual a 0 significa que o modelo não explica a variabilidade dos dados, enquanto valores negativos (possíveis em alguns casos) indicam que o modelo é pior do que uma previsão baseada apenas na média.

- **MAPE (Mean Absolute Percentage Error)** - Expressa o erro médio em termos percentuais, calculando a razão entre a diferença absoluta do valor predito e o valor real. Quanto menor o valor do MAPE, mais preciso é o modelo. Essa métrica é intuitiva e fácil de comunicar, porém pode apresentar limitações quando os valores reais são muito próximos de zero, o que pode distorcer a interpretação.

Em conjunto, essas métricas fornecem uma visão abrangente do desempenho do modelo, permitindo avaliar não apenas a magnitude do erro, mas também a capacidade explicativa e a robustez das previsões.


In [7]:
df_resul = df_metricas.drop(['RMSE', 'R² Ajustado', 'RMSLE'], axis=1)
df_resul.style.format("{:.3f}", subset = df_resul.select_dtypes(include="number").columns).background_gradient(cmap='Blues').set_table_styles(
    [{
        'selector': 'th', # 'th' seleciona as células do cabeçalho
        'props': [('text-align', 'center')]
    }]
).set_properties(subset=['Modelo'],**{'text-align': 'center'})

,Modelo,MAE,MSE,R²,MAPE (%)
0,Regressão Linear,6.164,60.736,0.310,27.121
1,LightGBM,5.727,53.115,0.397,25.053
2,Random Forest Regressor,5.962,57.767,0.344,26.115
3,XGBoost,5.859,56.103,0.363,25.541
4,CatBoost,5.739,53.551,0.392,25.102


## 2.1 Explicabilidade do Modelo
Os resultados dos modelos foram comparados buscando aquele que apresenta-se menor **MAE**,**MSE** e **MAPE**. Nesse sentido os modelos que tiveram melhor resultado foram 
1. Lightgbm
2. CatBoost

Ambos apresentaram **MAE** de 5,7, ou seja, esse valor representa a média da distância do valor real e o valor predito. No quesito **MSE**, quanto menor melhor e nisso o Lightgbm e CatBoost foram os melhores. Em relação ao **MAPE**, quanto menor, mais preciso é o modelo. O Lightgbm tem **MAPE = 25,1 %**, isso significa que, o valor que o modelo prever está errado por 25,1% do valor real. o mesmo vale para o CatBoost que tem **MAPE = 25,41%**.

Uma feature muito importante ficou de fora na hora da construção dos modelos: o horário em que o pedido foi feito. Essa informação é crucial para um serviço de delirevy, pois apartir dela, rotas de entragas podem ser planejadas, tipos de veículo que melhor se adequem ao trânsito naquele momento. A informação do horário não existem no nosso banco de dados, comprometendo muito qualquer análise mais criteriosa.

Entretando mesmo com a ausência deste informação foi possível construir um modelo preditivo que ajuda em futuros insights para a empresa.

<div style="background-color: #103778; 
            color: white; 
            padding: 10px; 
            text-align: center;
            font-size: 20px;">
    <h1>3. Ajuste de Hiperparâmetros com o Optuna</h1>
</div>

Algo que posso ser feito para melhor o resultado do modelo é um **ajuste de hiperpâmetros**, isso otimiza o desempenho de um modelo de aprendizado de máquinas.

Diferentemente dos parâmetros que são aprendidos automaticamente pelo modelo durante o treinamento, os hiperparâmetros são configurações externas definidas pelo cientista de dados antes do início do treinamento. O ajuste (ou tuning) de hiperparâmetros visa encontrar a combinação ideal dessas configurações para que o modelo possa generalizar da melhor forma possível para dados não vistos.

Vamos realizar o tuning nos modelos **Lightgbm** e **CatBoost** usando o **Optuna**.

**Optuna** é um framework de otimização de hiperparâmetros em open source para Python, que utiliza métodos eficientes, como a otimização bayesiana, para encontrar a melhor combinação de configurações para o modelo de machine learning. Ele funciona em três passos.
1. Definir uma função objetivo - Esta função encapsula o treinamento do modelo e a avaliação do desempenho. Ela recebe um objeto `trial` como parâmetro, que é usado para sugerir hiperparâmetros. A função deve retornar a métrica de desempenho que desejamos otimizar, no nosso caso será **MAE**.

2. Cria um objeto de estudo (`study`) - Ele gerencia todo o processo de otimização.

3. Execução da Otimização - O métodp `optimize()` no objeto `study`, passando a função objetivo e o número de tentativas.

Após esse processo, treinamos novamente o modelo mas dessa vez com os melhores parâmetros obtido pela função `ojective`. 

## 3.1 Lightgbm Ajustado

In [8]:
df_lgbm, model_lgbm, study_lgbm = ml.optuna_regression(
    X_train, y_train, X_test, y_test,
    modelo="lightgbm",
    n_trials=50
)

## 3.2 CatBoost Ajustado

In [ ]:
# Precesso leva aproximadamente 3 minutos
df_cat, model_cat, study_cat = ml.optuna_regression(
    X_train, y_train, X_test, y_test,
    modelo="catboost",
    n_trials=50
)

In [10]:
df_resultados = pd.concat([df_cat, df_lgbm])
df_resultados

,Modelo,MAE,MSE,R²,MAPE (%)
0,CatBoost,5.667623,54.116381,0.385579,24.181042
0,LightGBM,5.712131,52.857508,0.399872,25.012955


Ambos os modelos apresentaram desempenho semelhante, com erro médio em torno de 6 minutos. O CatBoost se destacou levemente em métricas de erro absoluto e percentual, enquanto o LightGBM apresentou melhor capacidade explicativa. Considerando o contexto de negócio, onde a precisão média da previsão é mais relevante do que a explicação da variância, o CatBoost foi selecionado como modelo final.

<div style="background-color: #103778; 
            color: white; 
            padding: 10px; 
            text-align: center;
            font-size: 20px;">
    <h1>Conclusão</h1>
</div>

Este projeto serviu como estudo de caso para construção de um modelo utilizando técnicas de aprendizado de máquina, capaz de prever os tempos de entrega dos pedidos feito para serviços de delirevy, primeiramente foi feito uma limpeza dos dados quando foi constatado discrepâncias nos dados de latitude e longitude, que apresentavam valores negativos e próximo de zero, valores estes que estavam totalmente fora do escopo real dos dados.

Após a limpeza foi realizado uma análise explorátoria dos dados fornecendo insigths relavantes para o setor de delirevy, como quantidade de tipo de pedido, taxa de avaliação dos delivery, distância entre o restaurante e o local de entrega, idade e veículo dos entregadores (delivery). 

A construção de features veio em seguida, passo essencial na construção do modelo, aqui foi analisado dados que fossem relevantes para o objetivo deste projeto. Durante essa etapa foi verificado que os dados **não continham informações sobre horário ou turno do dia que o pedido foi feito**. Informação essa de extrema importância para um serviço de delivery, já que, por meio dela é possível criar um planejamento de rota, tipo de veículo mais adequado para o tipo de pedido feito, movimentação do trânsito no momento da entrega.

O estudo seguiu a para o treinamento do modelo, foi usado 5 modelos, todos com abordagens e métodos diferentes:
- Linear Regression
- Lightgbm
- Random Forest Regression
- XGBoost
- Catboost

Os resultados obtidos foram comparados buscando aquele que melhor atendesse ao objetivo final. Lightgbm e XGBoost tiveram os melhores resultados. Com a intenção de contornar o problema identificado da ausente do horário do pedido, foi feito um ajuste de hiperpâmetros nos modelos que tiveram melhor resultado. Porém, mesmo com essa estrágia a alteração no resultado foi de pouca valia, já houve apenas uma leve mudança de valores. 

Como recomendação final para este projeto seria conversar com o setor de negócio responsável pelo armazenamento dos dados e com o setor estrátegico da empresa para discutir planos de ação para melhorar a acurácia dos dados, planos de avaliação de rotas de acordo com o trânsito no momento que o pedido foi feito até a entrega, para que haja possibilidade de buscar novas variáveis ou novas variáveis derivadas que possam ser adicionadas aos modelos aumentando sua precisão.
